In [73]:
#!cp 0-sequential.py 1-input.py
!chmod +x *.py

In [72]:
#!/usr/bin/env python3

#build_model = __import__('0-sequential').build_model
import tensorflow as tf
import tensorflow.keras as K


def build_modell(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    for i in range(len(layers)):
        init = (tf.contrib.layers.
            variance_scaling_initializer(mode="FAN_AVG"))
        freg = K.layers.ActivityRegularization(l2=lambtha)
        layer = K.layers.Dense(layers[i], input_dim=nx,
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=keep_prob)
            model.add(dropped)
    return (model)

def build_model(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    init = (tf.contrib.layers.
            variance_scaling_initializer(mode="FAN_AVG"))
    freg = K.regularizers.l2(l=lambtha)
    layer = K.layers.Dense(layers[0], input_shape=(nx,),
                           activation=activations[0],
                           kernel_initializer=init,
                           kernel_regularizer=freg)
    model.add(layer)
    dropped = K.layers.Dropout(rate=keep_prob)
    model.add(dropped)
    for i in range(1, len(layers)):
        init = (tf.contrib.layers.
            variance_scaling_initializer(mode="FAN_AVG"))
        freg = K.regularizers.l2(l=lambtha)
        layer = K.layers.Dense(layers[i], input_shape=(nx,),
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=keep_prob)
            model.add(dropped)
    return (model)

if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_134 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_78 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_135 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_79 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_136 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'kernel/Regularizer_107/add:0' shape=() dtype=float32>, <tf.Tensor 'kernel/Regularizer_109/add:0' shape=() dtype=float32>, <